In [1]:
from llama_index.llms import Ollama
from llama_index.llms import ChatMessage

In [2]:
import llama_index

llama_index.__version__

'0.8.42'

In [2]:
llm = Ollama(model="llama2")

In [3]:
messages = [ChatMessage(role="user", content="Hello, how are you?"), ChatMessage(role="system", content="You are poet, who write funny poems!")]

In [4]:
out = llm.stream_chat(messages)

for ele in out:
    print(ele.delta, end="")

 Thank you! *adjusts sunglasses* I'm doing great, thanks for asking! *winks* But enough about me, what can I do for you today? Do you have a favorite poem of mine that you'd like to share or discuss? Or perhaps you have a topic in mind for a new poem? Just let me know! 😁

In [ ]:
resp_stream = llm.stream_complete("Who is Paul Graham?")

for r in resp_stream:
    print(r.delta, end="")

In [ ]:
import random

def random_response(message, history):
    return random.choice(["Yes", "No"])

In [ ]:
import gradio as gr

gr.ChatInterface(random_response).launch()

In [ ]:

def my_chat_bot(message, history):
    msg = [
        ChatMessage(role="system", content="You are Intelligent AI Chatbot! You are Honest, Helpful and Harmless!"),
        ChatMessage(role="user", content=message)]

    resp_bot = llm.chat(msg)
    
    return resp_bot.dict()['message']['content']

In [ ]:
import gradio as gr

demo = gr.ChatInterface(my_chat_bot)

In [ ]:
demo.queue().launch()

In [6]:
from llama_index.tools import FunctionTool
from llama_index.agent import ReActAgent

In [9]:
from datetime import datetime
from llama_index.tools.tool_spec.base import BaseToolSpec

class DateToolSpec(BaseToolSpec):
    
    spec_functions = ["get_today_date"]
    
    def get_today_date(self):
        """ Tool Returns today's date in dd-mm-yyyy format
        """
        return datetime.today().strftime('%d-%m-%Y')

# get_today_date()

In [11]:
date_spec = DateToolSpec()

tools = date_spec.to_tool_list()

In [9]:
# get_today_date_tool = FunctionTool.from_defaults(fn=get_today_date, name="get_today_date", description="Use this tool to get today's date in dd-mm-yyyy format")
# tools = [get_today_date_tool]


In [5]:
def agent(message, history):
    agent = ReActAgent.from_tools(tools=tools, llm=llm, verbose=True)
    response = agent.chat(message)
    return response

print(agent("Give me todays date", []))

NameError: name 'ReActAgent' is not defined

In [12]:
from llama_index.tools import BaseTool, ToolMetadata, ToolOutput

class DateTool(BaseTool):
    @property
    def metadata(self):
        return ToolMetadata(
            description="Tool Returns today's date in dd-mm-yyyy format",
            name="get_today_date",
        )

    def __call__(self) -> ToolOutput:
        return ToolOutput(datetime.today().strftime('%d-%m-%Y'))

In [18]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply, name="multiply", description="Use this tool to multiply two integers")


In [19]:
def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add, name="add", description="Use this tool to add two integers")


In [21]:
def agent(message, history):
    agent = ReActAgent.from_tools(tools=[multiply_tool, add_tool], llm=llm, verbose=True)
    response = agent.chat(message)
    return response

print(agent("Use the provided tools to solve the problem. Problem: What is 10*2? Calculate step by step", []))

Thought: I need to use a tool to help me answer the question.
Action: tool name (multiply)
Action Input: {'a': 10, 'b': 2}


KeyError: 'tool name (multiply)'